In [ ]:
! pip install transformers==4.28.0 datasets


# Image classification

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load dataset

In [ ]:
from datasets import load_dataset

food = load_dataset("juanfengyun/SEEDTrain", split="train[:325]")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset

ds = load_dataset("/content/drive/MyDrive/MFCC/BB1", split="train")

Split the dataset's `train` split into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [ ]:
ds

In [ ]:
ds.data

In [ ]:
ds = ds.train_test_split(test_size=0.2)

In [ ]:
ds


In [ ]:
labels = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

Now you can convert the label id to a label name:

In [ ]:
labels

In [ ]:
id2label

## Preprocess

The next step is to load a ViT image processor to process the image into a tensor:

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

Then create a preprocessing function to apply the transforms and return the `pixel_values` - the inputs to the model - of the image:

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
ds = ds.with_transform(transforms)

In [ ]:
testDs = ds["test"]

In [ ]:
testDs

In [ ]:
testDs = load_dataset("/content/drive/MyDrive/Colab Notebooks/SpeechClassification/test")

In [ ]:
testDs

In [ ]:
prepared_Test = testDs.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Evaluate

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import evaluate


In [ ]:
metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metrics.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you set up your training.

In [ ]:
!pip install wandb
import wandb

wandb.login()

## Train

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
pip
install - -upgrade
accelerate

In [ ]:
pip
install
transformers == 4.28
.0

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_MFCC_VITmodelBBMetrics",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=80,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    #  report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds["train"],
    # eval_dataset=food["test"],
    eval_dataset=ds["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
wandb.finish()

In [ ]:
trainer.save_state()

In [ ]:
trainer.state.log_history

In [ ]:
type(trainer.state.log_history[0])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.DataFrame(trainer.state.log_history)
data


In [ ]:
data.dropna(subset=['eval_accuracy', 'eval_loss'])

In [ ]:
from matplotlib import pyplot as plt

_df_13['eval_accuracy'].plot(kind='line', figsize=(8, 4), title='eval_accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt

_df_15['eval_samples_per_second'].plot(kind='line', figsize=(8, 4), title='eval_samples_per_second')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tight_layout()

In [ ]:
trainer.push_to_hub()

## Inference

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds["train"],
    # eval_dataset=food["test"],
    eval_dataset=ds["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)


In [ ]:
ds['test']

In [ ]:
trainer.evaluate(ds['test'])

In [ ]:
labels1 = prepared_Test["train"].features["label"].names
label2id1, id2label1 = dict(), dict()
for i, label in enumerate(labels1):
    label2id1[label] = str(i)
    id2label1[str(i)] = label

In [ ]:
image = ds["image"]
image[2]

In [ ]:
type(image)

In [ ]:
for i in range(10):
    image[i]

In [ ]:
labels1[0]

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_MFCC_VITmodel")
for i in range(110):
    list = classifier(image[i])
list
# for img in image:
#   classifier(img)

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("my_awesome_food_model")
inputs = image_processor(image, return_tensors="pt")

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("my_awesome_food_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]